In [2]:
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
import pandas as pd
import numpy as np

#train_data_source = './exercise_data/human_dna_train_small.csv'
train_data_source = './exercise_data/human_dna_train_split_5_95.csv'
val_data_source = './exercise_data/human_dna_validation_split.csv'
test_data_source = './exercise_data/human_dna_test_split.csv'

train_df = pd.read_csv(train_data_source, header=0)
test_df = pd.read_csv(test_data_source, header=0)

train_seq = train_df['sequences']
train_label = train_df['labels']
test_seq = test_df['sequences']
test_label = test_df['labels']

# Preprocess
tk = text.Tokenizer(char_level=True)
tk.fit_on_texts(train_seq)

train_seq_tok = tk.texts_to_sequences(train_seq)
test_seq_tok = tk.texts_to_sequences(test_seq)

train_seq = np.array(train_seq)
train_seq_tok = np.array(train_seq_tok)
train_label = np.array(train_label)
test_seq = np.array(test_seq)
test_seq_tok = np.array(test_seq_tok)
test_label = np.array(test_label)

print('train_seq shape:', train_seq.shape)
print('test_seq shape:', test_seq.shape)

train_seq shape: (29420,)
test_seq shape: (33333,)


In [3]:
train_label[train_label<0]=0
test_label[test_label<0]=0

In [4]:
train_label

array([1, 1, 1, ..., 0, 0, 0])

In [5]:
train_seq_tok = train_seq_tok - np.ones(shape=train_seq_tok.shape)
test_seq_tok = test_seq_tok - np.ones(shape=test_seq_tok.shape)

In [6]:
train_seq_tok

array([[0., 3., 1., ..., 1., 0., 1.],
       [0., 2., 2., ..., 0., 0., 0.],
       [1., 2., 2., ..., 1., 2., 0.],
       ...,
       [2., 0., 0., ..., 1., 1., 2.],
       [1., 3., 1., ..., 2., 1., 1.],
       [2., 0., 0., ..., 1., 1., 1.]])

In [7]:
train_seq_tok_split = np.hsplit(train_seq_tok, 2)

In [8]:
train_seq_tok.shape

(29420, 398)

In [9]:
train_seq_tok_split[0].shape

(29420, 199)

In [10]:
train_seq_tok[0]

array([0., 3., 1., 1., 2., 1., 2., 2., 0., 0., 1., 1., 2., 3., 1., 1., 3.,
       0., 0., 2., 0., 2., 2., 1., 0., 2., 2., 0., 3., 1., 0., 2., 2., 1.,
       3., 2., 1., 0., 1., 2., 2., 2., 1., 1., 3., 0., 1., 0., 0., 2., 3.,
       1., 1., 1., 0., 0., 3., 0., 0., 2., 1., 2., 3., 1., 0., 2., 2., 0.,
       2., 0., 0., 2., 1., 3., 1., 2., 2., 2., 1., 1., 2., 1., 1., 1., 1.,
       3., 1., 2., 0., 1., 0., 0., 3., 3., 1., 2., 1., 0., 3., 1., 2., 0.,
       0., 1., 2., 0., 2., 0., 2., 2., 3., 1., 2., 0., 1., 1., 1., 1., 3.,
       1., 3., 1., 0., 2., 1., 2., 2., 2., 2., 0., 2., 1., 3., 0., 2., 1.,
       1., 2., 2., 2., 1., 1., 2., 3., 1., 2., 3., 1., 1., 1., 1., 1., 3.,
       0., 2., 2., 3., 1., 2., 1., 1., 1., 1., 1., 1., 3., 1., 1., 2., 3.,
       1., 2., 3., 0., 3., 0., 0., 0., 3., 0., 3., 0., 2., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 2., 3., 3., 1., 1., 1., 2., 2., 1., 0., 1., 3.,
       3., 0., 3., 3., 3., 0., 3., 1., 1., 3., 1., 1., 0., 2., 2., 2., 3.,
       1., 0., 0., 3., 3.

In [11]:
test_seq_tok_split = np.hsplit(test_seq_tok, 2)

In [19]:
train_seq_tok_reshape = np.expand_dims(train_seq_tok, axis=2)
test_seq_tok_reshape = np.expand_dims(test_seq_tok, axis=2)

In [20]:
train_seq_tok_reshape.shape

(29420, 398, 1)

In [23]:
from keras.layers import Dropout, concatenate, Input
import keras as K

# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

max_features = 20000
batch_size = 256
epochs = 100
print('Build model...')
model = Sequential()
# model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Train...')
model.fit(train_seq_tok_reshape, train_label, batch_size=batch_size, epochs=epochs, validation_data=(test_seq_tok_reshape, test_label), class_weight={0: 0.5, 1: 0.95})
score, acc = model.evaluate(test_seq_tok_reshape, test_label, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Train on 29420 samples, validate on 33333 samples
Epoch 1/100
29420/29420 [==============================] - 192s 7ms/step - loss: 0.1721 - accuracy: 0.9429 - val_loss: 0.1002 - val_accuracy: 0.9969
Epoch 2/100
29420/29420 [==============================] - 183s 6ms/step - loss: 0.1596 - accuracy: 0.9500 - val_loss: 0.0871 - val_accuracy: 0.9969
Epoch 3/100
29420/29420 [==============================] - 185s 6ms/step - loss: 0.1595 - accuracy: 0.9500 - val_loss: 0.0972 - val_accuracy: 0.9969
Epoch 4/100
29420/29420 [==============================] - 183s 6ms/step - loss: 0.1594 - accuracy: 0.9500 - val_loss: 0.0912 - val_accuracy: 0.9969
Epoch 5/100
 7168/29420 [======>.......................] - ETA: 1:43 - loss: 0.1618 - accuracy: 0.9488

KeyboardInterrupt: 